Heatmap of the first thirty questions to all users (based on 10M records).

The standard questions and the bundles are easy to identify.  The patterns seem to continue (in increments of 30) but in much smaller quantities.  If you play with this, pay attention to changes in the scale, it changes dramatically.

I wanted to get this further along but didn't want to wait any longer.

Happy New Year!

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', nrows=10_000_000)
df = df.loc[df.content_type_id == 0].reset_index(drop=True) # drop lectures
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
df['counter'] = 1
df['user_ques_num'] = df.groupby('user_id').cumcount()+1

In [ ]:
def questions_heatmap(df, start_ques=0, num_of_ques=30, start_content_id_rank=0, num_of_content_ids=70, figsize=(10,10), show_avg=False, scb=.99):

    df_stats = df.groupby(['content_id'])[['counter']].sum().sort_values(['counter'], ascending=False).reset_index().reset_index().rename(columns={'index':'content_id_rank', 'counter':'content_id_attempts'})
    df2 = df.merge(df_stats[['content_id', 'content_id_rank', 'content_id_attempts']], left_on='content_id', right_on='content_id', how='left')

    df2 = df2[((df2['user_ques_num']>= start_ques) & (df2['user_ques_num']<= start_ques + num_of_ques))] # limit user questions
    df2 = df2[((df2['content_id_rank']>=start_content_id_rank) & (df2['content_id_rank']<=(start_content_id_rank+num_of_content_ids)))].reset_index()

    df2 = df2.groupby(['content_id', 'user_ques_num', 'content_id_rank', 'content_id_attempts'])[['counter', 'answered_correctly']].sum() # .sort_values(['counter'], ascending=False)
    df2['cont_set_ques_pct'] = df2['answered_correctly'] / df2['counter']
    df2.drop(['answered_correctly'], axis=1, inplace=True)
    df2 = pd.DataFrame(df2.unstack(level=1)).sort_values(['content_id_rank'], ascending=[True]).fillna(0)
    
    new_y_axis_tms = zip(list(df2.index.get_level_values(0)), list(df2.index.get_level_values(1)), list(df2.index.get_level_values(2)))
    
    cols = [col for col in df2.columns if col[0] == 'cont_set_ques_pct']
    if show_avg:
        x = df2[cols].to_numpy()
        y = np.array([f"{w:.2f}".lstrip('0') for w in x.reshape(x.size)])
        y = y.reshape(x.shape)
    else:
        y = False
    df2.drop(cols, axis=1, inplace=True)
    df2.columns = list(range(cols[0][1], df2.shape[1] + cols[0][1]))

    fig = plt.figure(figsize=figsize)
    ax1 = plt.subplot2grid((20, 20), (0, 0), colspan=20, rowspan=20)
    comma_fmt = FuncFormatter(lambda x, p: format(int(x), ','))
    sns.heatmap(df2, 
                ax=ax1, 
                cmap='gist_heat_r', 
                square=True,
                annot=y, 
                cbar_kws={"shrink": scb, 'format': comma_fmt},
                annot_kws={"fontsize":8}, 
                fmt='');
    plt.xlabel("question number", fontsize=15)
    plt.ylabel("( Content id  /  Times Asked Rank  /  # Times Asked )", fontsize=15)
    
#     ax1.set_yticklabels(new_y_axis_tms)
    ax1.set_title('Riiid Question Heatmap\n', fontsize=20)
    plt.yticks(rotation=0)
    plt.show()

In [ ]:
questions_heatmap(df, start_ques=0, num_of_ques=30, start_content_id_rank=0, num_of_content_ids=44, figsize = (10,15), show_avg=False, scb=.8 )

Below zooms out showing the first two hunderd questions and content id's using the same scale as above.

In [ ]:
questions_heatmap(df, start_ques=0, num_of_ques=200, start_content_id_rank=0, num_of_content_ids=200, figsize=(20,25), show_avg=False, scb=.65)